In [ ]:
# !apt-get install git-lfs
# !git lfs install

In [ ]:
# !pip install torchvision
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100

In [ ]:
# install torchvision for kaggle
# !apt-get install -y python-dev pkg-config
# !apt-get install -y libavformat-dev libavcodec-dev libavdevice-dev libavutil-dev libswscale-dev libswresample-dev libavfilter-dev
# !pip install av

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [ ]:
import torchvision

In [ ]:
# TODO seperate by label folder
# os.listdir('../input/drowsiness-dataset-fold1-1/Fold1_part1/05')

In [ ]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

batch_size=1
sequence_length=8
initial_prefetch_size=16
# /content/drive/MyDrive/DALI_extra/db/video/sintel/video_files
video_directory="../input/drowsiness-dataset-fold1-1/Fold1_part1"
shuffle=True
n_iter=6
print(video_directory)

In [ ]:
!ls video_directory

In [ ]:
class VideoPipe(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, data, shuffle):
        super(VideoPipe, self).__init__(batch_size, num_threads, device_id, seed=42)
#         file = 
        self.input = ops.VideoReader(device="gpu", file_root=data, sequence_length=sequence_length,
                                shard_id=0, num_shards=1,
                                skip_vfr_check=True
                                file_list=f'{file} {label} [{start_frame} [[end_frame]]]'
                                random_shuffle=shuffle, initial_fill=initial_prefetch_size, file_list_frame_num=True, step=2, stride=2)


    def define_graph(self):
        output, labels = self.input()
        #crop output to be in the same size
        return (output, labels)



In [ ]:
pipe = VideoPipe(batch_size=batch_size, num_threads=2, device_id=0, data=video_directory, shuffle=shuffle)
pipe.build()

for i in range(n_iter):
    sequences_out, labels = pipe.run()
    #   print(pipe.run())
    sequences_out = sequences_out.as_cpu().as_array()
    labels = labels.as_cpu().as_array()
    print("sequence=", sequences_out.shape)
    print("labels: ", labels)

In [ ]:
# # %%time
# video_path = '../input/drowsiness-dataset-fold1-1/Fold1_part1/01/0.mov'
# import cv2
# def get_video_meta(video_path):
#     cap=cv2.VideoCapture(video_path)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
# #     print(f"video path = {video_path}")
# #     print(f"fps ={fps}")
#     duration = frame_count/fps
    
#     return {'fps':fps,'frame_count':frame_count, 'duration':duration }

# def get_path(x):
#     files = os.listdir(x)
#     return list(map(lambda file: os.path.join(x, file), files))